**Database Name: Logistics Management**

This database is designed to manage and track various aspects of a logistics and delivery system, focusing on the details of drivers, vehicles, deliveries, locations, traffic data, and performance metrics. It provides a centralized structure to efficiently store and retrieve information about each component involved in the logistics process. The database can be used for tracking deliveries, optimizing routes, managing resources, and analyzing performance.

**Tables/Entities**

1. Drivers
 * Description: Represents drivers who handle deliveries.
 * Attributes:
    * DriverID: Unique identifier for each driver.
    * Name: Full name of the driver.
    * LicenseNo: Driver’s license number.
    * PhoneNumber: Contact number for the driver.
 * Purpose: Stores personal and identification information about drivers.
 * Relationship: One driver can handle multiple deliveries
2. Vehicles
  * Description: Represents vehicles used for deliveries.
  * Attributes:
    * VehicleID: Unique identifier for each vehicle.
    * VehicleType: Type of vehicle (e.g., truck, van).
    * Capacity: The carrying capacity of the vehicle.
  * Purpose: Holds details about the fleet of vehicles available for deliveries.
  * Relationship: Each vehicle can be associated with multiple deliveries.
3. Deliveries
 * Description: Represents individual delivery assignments.
 * Attributes:
    * DeliveryID: Unique identifier for each delivery.
    * DriverID: References the driver assigned to the delivery.
    * VehicleID: References the vehicle used for the delivery.
    * DeliveryDate: The date of the delivery.
    * DeliveryStatus: Current status of the delivery (e.g., pending, completed).
 * Purpose: Links drivers, vehicles, and delivery schedules.
 * Relationships:
    * One-to-many with drivers and vehicles.
    * Many-to-one with start and end locations.
    * Many-to-many with  locations.
4. Locations
 * Description: Represents places involved in deliveries (e.g., warehouses, customer addresses).
 * Attributes:
    * LocationID: Unique identifier for each location.
    * LocationName: Name of the location (e.g., "Warehouse A").
    * Address: Physical address of the location.
 * Purpose: Stores information about all locations associated with deliveries.
 * Relationships:
    * Many-to-many with deliveries .
    * One-to-many with traffic data for real-time or historical traffic tracking.
    * One-to-Many with routes: One location can be part of multiple routes, whether as a starting or ending point.
5. TrafficData
 * Description: Tracks traffic conditions for specific locations at given times.
 * Attributes:
    * TrafficDataID: Unique identifier for each traffic record.
    * LocationID: References the location associated with the traffic data.
    * TrafficCondition: Description of the traffic state (e.g., "Clear," "Heavy").
    * TimeStamp: Time when the traffic condition was recorded.
 * Purpose: Captures real-time traffic data to help optimize delivery routes.
 * Relationship: Each traffic entry is tied to a location.
6. DeliveryMetrics
 * Description: Stores performance metrics for individual deliveries.
 * Attributes:
    * MetricID: Unique identifier for each metric record.
    * DeliveryID: References the associated delivery.
    * OnTime: Boolean indicating whether the delivery was on time.
    * DelayReason: Reason for delay (if applicable).
 * Purpose: Provides insights into delivery performance and reasons for any delays.
 * Relationship:  One-to-one for performance tracking. Tracks performance metrics, such as on-time status and delays, linked one-to-one with deliveries.
7. RouteMetrics
 * Description: Stores performance data for specific routes between locations.
 * Attributes:
     * RouteMetricID: Unique identifier for each route metric record.
     * StartLocationID: References the starting location.
     * EndLocationID: References the ending location.
     * EstimatedTime: Estimated time for the route.
     * ActualTime: Actual time taken to complete the route.
 * Purpose: Tracks and compares estimated vs. actual times for delivery routes, aiding in route optimization.

8. DeliveryLocations
 * Description: Links deliveries to locations with additional details about location type.
 * Attributes:
     * DeliveryLocationID: Unique identifier for each delivery-location association.
     * DeliveryID: References the associated delivery.
     * LocationID: References the associated location.
     * LocationType: Indicates the role of the location in the delivery (e.g., "start," "end," "stop").
 * Purpose: Models the many-to-many relationship between deliveries and locations, providing context for each location's role in a delivery.



**Relations**

1. Relationships Between Entities Deliveries ↔ Drivers

    * Relationship Type: One-to-Many
    * A driver (DriverID) can handle many deliveries, but each delivery is assigned to one driver.
    * Foreign Key: DriverID in Deliveries references DriverID in Drivers.

2. Deliveries ↔ Vehicles

* Relationship Type: One-to-Many
* A vehicle (VehicleID) can be used for multiple deliveries, but each delivery uses one vehicle.
* Foreign Key: VehicleID in Deliveries references VehicleID in Vehicles.


 3. Deliveries ↔ Locations
* Relationship Type: Many-to-Many
* Description: A delivery can pass through multiple locations, and a location can be part of multiple deliveries. This is managed via the DeliveryLocations junction table.
* Foreign Keys:
     * DeliveryID in DeliveryLocations references DeliveryID in Deliveries.
     * LocationID in DeliveryLocations references LocationID in Locations.

4. Deliveries ↔ DeliveryMetrics
 * Relationship Type: One-to-One
 * Description: Each delivery has one set of associated delivery metrics.
 * Foreign Key: DeliveryID in DeliveryMetrics references DeliveryID in Deliveries.

5. Locations ↔ TrafficData
  * Relationship Type: One-to-Many
  * Description: A location (LocationID) can have multiple traffic data entries recorded over time.
  * Foreign Key: LocationID in TrafficData references LocationID in Locations.

 6. Locations ↔ RouteMetrics
  * Relationship Type: One-to-Many
  * Description: A location can serve as the start or end point for multiple routes.
  * Foreign Keys:
      * StartLocationID in RouteMetrics references LocationID in Locations.
      * EndLocationID in RouteMetrics references LocationID in Locations.



In [2]:
import sqlite3
import random
import string
from datetime import datetime, timedelta
import pandas as pd

In [3]:
# Connect to SQLite database (or create it if it doesn't exist)
# sqlite3.connect() establishes a connection to the specified SQLite database.
# If the database does not exist, it will be created.
conn = sqlite3.connect('logistics.db')

# Create a cursor object, which is used to execute SQL commands
cursor = conn.cursor()



In [ ]:
# Create the 'Drivers' table if it does not already exist
# This table stores information about drivers, such as their ID, name, license number, and phone number.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Drivers (
        DriverID INTEGER PRIMARY KEY,       -- The unique identifier for each driver
        Name TEXT NOT NULL,                  -- The driver's name (cannot be NULL)
        LicenseNo TEXT NOT NULL,             -- The driver's license number (cannot be NULL)
        PhoneNumber TEXT NOT NULL            -- The driver's phone number (cannot be NULL)
    )
''')



In [ ]:
# Create the 'Vehicles' table if it does not already exist
# This table stores information about vehicles, including their ID, type, and capacity.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Vehicles (
        VehicleID INTEGER PRIMARY KEY,       -- The unique identifier for each vehicle
        VehicleType TEXT NOT NULL,           -- The type of the vehicle (e.g., truck, van, etc.)
        Capacity INTEGER NOT NULL            -- The capacity of the vehicle (e.g., number of items it can carry)
    )
''')



In [ ]:
# Create the 'Deliveries' table if it does not already exist
# This table stores delivery information such as the assigned driver and vehicle, delivery date, and status.
# Foreign keys are used to reference the 'Drivers' and 'Vehicles' tables.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Deliveries (
        DeliveryID INTEGER PRIMARY KEY,      -- The unique identifier for each delivery
        DriverID INTEGER,                    -- The ID of the driver assigned to the delivery
        VehicleID INTEGER,                   -- The ID of the vehicle assigned to the delivery
        DeliveryDate TEXT NOT NULL,           -- The date of the delivery (cannot be NULL)
        DeliveryStatus TEXT NOT NULL,         -- The status of the delivery (e.g., pending, completed)
        FOREIGN KEY (DriverID) REFERENCES Drivers(DriverID),  -- Establishes a foreign key relationship with the Drivers table
        FOREIGN KEY (VehicleID) REFERENCES Vehicles(VehicleID) -- Establishes a foreign key relationship with the Vehicles table
    )
''')



In [ ]:
# Create the 'Locations' table if it does not already exist
# This table stores information about various locations, including their names and addresses.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Locations (
        LocationID INTEGER PRIMARY KEY,       -- The unique identifier for each location
        LocationName TEXT NOT NULL,            -- The name of the location (cannot be NULL)
        Address TEXT NOT NULL                  -- The address of the location (cannot be NULL)
    )
''')


In [ ]:

# Create the 'TrafficData' table if it does not already exist
# This table stores traffic data for specific locations, including the traffic condition and timestamp.
# Foreign key references the 'Locations' table.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS TrafficData (
        TrafficDataID INTEGER PRIMARY KEY,    -- The unique identifier for each traffic data entry
        LocationID INTEGER,                    -- The ID of the location associated with the traffic data
        TrafficCondition TEXT NOT NULL,        -- The condition of traffic (e.g., clear, heavy, etc.)
        TimeStamp TEXT NOT NULL,               -- The timestamp of when the traffic data was recorded
        FOREIGN KEY (LocationID) REFERENCES Locations(LocationID) -- Establishes a foreign key relationship with the Locations table
    )
''')



In [4]:
# Question 1: Create the 'DeliveryMetrics' table if it does not already exist
# This table stores performance metrics for deliveries, such as whether they were on time and reasons for delays.
# Foreign key references the 'Deliveries' table.




In [ ]:
# Create the 'RouteMetrics' table if it does not already exist
# This table stores data on route performance, including estimated and actual delivery times for each route.
# Foreign keys reference the 'Locations' table for both start and end locations.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS RouteMetrics (
        RouteMetricID INTEGER PRIMARY KEY,    -- The unique identifier for each route metric entry
        StartLocationID INTEGER,               -- The ID of the starting location of the route
        EndLocationID INTEGER,                 -- The ID of the ending location of the route
        EstimatedTime TEXT NOT NULL,           -- The estimated time to complete the route (e.g., "1 hour")
        ActualTime TEXT NOT NULL,              -- The actual time taken to complete the route
        FOREIGN KEY (StartLocationID) REFERENCES Locations(LocationID), -- Establishes a foreign key relationship with the Locations table for the start location
        FOREIGN KEY (EndLocationID) REFERENCES Locations(LocationID) -- Establishes a foreign key relationship with the Locations table for the end location
    )
''')


In [ ]:

# Create the 'DeliveryLocations' table if it does not already exist
# This table links deliveries to their respective locations and the type of location (start, end, or stop).
# Foreign key references the 'Deliveries' and 'Locations' tables.
cursor.execute('''
    CREATE TABLE IF NOT EXISTS DeliveryLocations (
        DeliveryLocationID INTEGER PRIMARY KEY, -- The unique identifier for each delivery location entry
        DeliveryID INTEGER,                    -- The ID of the associated delivery
        LocationID INTEGER,                    -- The ID of the associated location
        LocationType TEXT CHECK(LocationType IN ('start', 'end', 'stop')) NOT NULL, -- Type of location (start, end, stop)
        FOREIGN KEY (DeliveryID) REFERENCES Deliveries(DeliveryID), -- Establishes a foreign key relationship with the Deliveries table
        FOREIGN KEY (LocationID) REFERENCES Locations(LocationID) -- Establishes a foreign key relationship with the Locations table
    )
''')



In [ ]:
# Commit changes to the database, saving the tables and schema definitions
conn.commit()

# Output a message indicating that the tables were created successfully
print("Tables created successfully.")

Tables created successfully.


In [ ]:
# Insert 50 rows into the 'Drivers' table
for _ in range(50):
    name = ''.join(random.choices(string.ascii_letters + string.digits, k=10))  # Random 10-character string for name
    license_no = ''.join(random.choices(string.ascii_letters + string.digits, k=8))  # Random 8-character string for license number
    phone_number = f"({random.randint(100, 999)}) {random.randint(100, 999)}-{random.randint(1000, 9999)}"  # Random phone number
    cursor.execute('''
        INSERT INTO Drivers (Name, LicenseNo, PhoneNumber)
        VALUES (?, ?, ?)
    ''', (name, license_no, phone_number))

In [ ]:
# Insert 50 rows into the 'Vehicles' table
for _ in range(50):
    vehicle_type = random.choice(["Truck", "Van", "Car", "Motorcycle"])  # Random vehicle type
    capacity = random.randint(1, 10)  # Random capacity between 1 and 10
    cursor.execute('''
        INSERT INTO Vehicles (VehicleType, Capacity)
        VALUES (?, ?)
    ''', (vehicle_type, capacity))

In [ ]:
# Insert 50 rows into the 'Locations' table
for _ in range(50):
    location_name = ''.join(random.choices(string.ascii_letters, k=8))  # Random 8-character string for location name
    address = ''.join(random.choices(string.ascii_letters + string.digits, k=15))  # Random 15-character string for address
    cursor.execute('''
        INSERT INTO Locations (LocationName, Address)
        VALUES (?, ?)
    ''', (location_name, address))


In [ ]:
# Insert 50 rows into the 'TrafficData' table
for _ in range(50):
    location_id = random.randint(1, 50)  # Random location ID between 1 and 50
    traffic_condition = random.choice(["Clear", "Heavy", "Moderate", "Congested"])  # Random traffic condition
    timestamp = (datetime.now() - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")  # Random date within the last year
    cursor.execute('''
        INSERT INTO TrafficData (LocationID, TrafficCondition, TimeStamp)
        VALUES (?, ?, ?)
    ''', (location_id, traffic_condition, timestamp))

In [ ]:
# Insert 50 rows into the 'Deliveries' table
for _ in range(50):
    driver_id = random.randint(1, 50)  # Random driver ID between 1 and 50
    vehicle_id = random.randint(1, 50)  # Random vehicle ID between 1 and 50
    delivery_date = (datetime.now() - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")  # Random delivery date within the last year
    delivery_status = random.choice(["Pending", "Completed", "In-Progress"])  # Random delivery status
    cursor.execute('''
        INSERT INTO Deliveries (DriverID, VehicleID, DeliveryDate, DeliveryStatus)
        VALUES (?, ?, ?, ?)
    ''', (driver_id, vehicle_id, delivery_date, delivery_status))



In [ ]:
# Insert 50 rows into the 'DeliveryMetrics' table
for _ in range(50):
    delivery_id = random.randint(1, 50)  # Random delivery ID between 1 and 50
    on_time = random.choice([True, False])  # Random on-time status
    delay_reason = ''.join(random.choices(string.ascii_letters, k=10)) if not on_time else None  # Random delay reason if not on time
    cursor.execute('''
        INSERT INTO DeliveryMetrics (DeliveryID, OnTime, DelayReason)
        VALUES (?, ?, ?)
    ''', (delivery_id, on_time, delay_reason))



In [ ]:
# Insert 50 rows into the 'RouteMetrics' table
for _ in range(50):
    start_location_id = random.randint(1, 50)  # Random start location ID between 1 and 50
    end_location_id = random.randint(1, 50)  # Random end location ID between 1 and 50
    estimated_time = f"{random.randint(1, 5)} hours"  # Random estimated time between 1 and 5 hours
    actual_time = f"{random.randint(1, 5)} hours"  # Random actual time between 1 and 5 hours
    cursor.execute('''
        INSERT INTO RouteMetrics (StartLocationID, EndLocationID, EstimatedTime, ActualTime)
        VALUES (?, ?, ?, ?)
    ''', (start_location_id, end_location_id, estimated_time, actual_time))



In [5]:
#Question 2: Insert 50 rows into the 'DeliveryLocations' table




In [6]:
#Question 3: Write an SQL query to update the PhoneNumber of a driver with DriverID = 5 in the Drivers table.
# Update the phone number for the driver with DriverID = 5


In [7]:
#Question 4: Write an SQL query to update the TrafficCondition in the TrafficData table to Congested for a LocationID of 5
#and also set the Capacity of the vehicle in the Vehicles table with VehicleID = 8 to 20



In [8]:
#Question 5: Write an SQL query to delete all deliveries with DeliveryStatus = 'Cancelled' from the Deliveries table.


In [9]:
#Question 6: Write an SQL query to select all columns from the Vehicles table where the VehicleType is 'Truck'.



In [10]:
#Question 7: Write an SQL query to select all columns from the Deliveries table where the DeliveryStatus is 'Completed' and the DriverID is 3.


In [11]:
#Question 8: Write an SQL query to select the count of deliveries grouped by DeliveryStatus from the Deliveries table.



In [12]:
#Question 9: Write an SQL query to select the DriverID, Name, and VehicleType for each delivery by joining the Deliveries, Drivers, and Vehicles tables.
